In [47]:
import torch
import pywt
from haar_pytorch import HaarForward, HaarInverse

haar = HaarForward()
ihaar = HaarInverse()

img = torch.randn(256, 128)
# wavelets1 = haar(img)
LL1, (LH1, HL1, HH1) = pywt.dwt2(img, 'haar')
wavelets2 = [LL1, LH1, HL1, HH1]
wavelets2 = torch.tensor(wavelets2)
print(LL1.shape, LH1.shape, HL1.shape, HH1.shape)


(128, 64) (128, 64) (128, 64) (128, 64)
0 torch.Size([128, 64]) torch.Size([]) tensor(6657.3433)
1 torch.Size([128, 64]) torch.Size([]) tensor(-2332.9092)
2 torch.Size([128, 64]) torch.Size([]) tensor(-1019.7534)
3 torch.Size([128, 64]) torch.Size([]) tensor(-19610.6348)


In [64]:
_LL1 = torch.randn(1,4,4256, 128)
# _LL1 = _LL1.unsqueeze(0).unsqueeze(0)
print(_LL1.shape)


torch.Size([1, 4, 4256, 128])


In [66]:
LL1, LH1, HL1, HH1 = _LL1[:,0,:,:], _LL1[:,1,:,:], _LL1[:,2,:,:], _LL1[:,3,:,:]
print(_LL1.shape)
E1 = torch.sqrt(torch.pow(LH1, 2)+torch.pow(HL1, 2)+torch.pow(HH1, 2))[0]

print(E1.shape)
        

torch.Size([1, 4, 4256, 128])
torch.Size([4256, 128])


In [ ]:
# print(wavelets1[0][0].shape, wavelets2[0][0][0].shape)
# for i in range(4):
    # print(i, wavelets1[0][i].shape, wavelets2[i][0][0].shape, (torch.sum(wavelets1[0][i] - wavelets2[i][0][0])))
# print(torch.sum(wavelets[0][0][0] - LL1[0][0]))
# print(wavelets[0][0][1] - LH1[0][0])
# print(wavelets[0][0][2] - HL1[0][0])
# print(wavelets[0][0][3] - HH1[0][0])

In [68]:
import torch

@torch.jit.sctipt
def blur_detect(Y, threshold):
    M, N = Y.shape
    haar = HaarForward()

    # Crop input image to be 3 divisible by 2
    Y = Y[0:int(M/16)*16, 0:int(N/16)*16]
    # add batch dimension
    Y = Y.unsqueeze(0).unsqueeze(0)
    
    # Step 1, compute Haar wavelet of input image
    _LL1 = haar(Y)
    LL1, LH1, HL1, HH1 = _LL1[:,0,:,:], _LL1[:,1,:,:], _LL1[:,2,:,:], _LL1[:,3,:,:]
    # Another application of 2D haar to LL1
    _LL2= haar(LL1[0]) 
    LL2, LH2, HL2, HH2 = _LL2[:,0,:,:], _LL2[:,1,:,:], _LL2[:,2,:,:], _LL2[:,3,:,:]
    # Another application of 2D haar to LL2
    _LL3 = haar(LL2[0])
    LL3 , LH3, HL3, HH3 = _LL3[:,0,:,:], _LL3[:,1,:,:], _LL3[:,2,:,:], _LL3[:,3,:,:]
    
    
    # Construct the edge map in each scale Step 2
    E1 = torch.sqrt(torch.power(LH1, 2)+torch.power(HL1, 2)+torch.power(HH1, 2))[0]
    E2 = torch.sqrt(torch.power(LH2, 2)+torch.power(HL2, 2)+torch.power(HH2, 2))[0]
    E3 = torch.sqrt(torch.power(LH3, 2)+torch.power(HL3, 2)+torch.power(HH3, 2))[0]
    
    M1, N1 = E1.shape
    print(E1.shape)

    # Sliding window size level 1
    sizeM1 = 8
    sizeN1 = 8
    
    # Sliding windows size level 2
    sizeM2 = int(sizeM1/2)
    sizeN2 = int(sizeN1/2)
    
    # Sliding windows size level 3
    sizeM3 = int(sizeM2/2)
    sizeN3 = int(sizeN2/2)
    
    # Number of edge maps, related to sliding windows size
    N_iter = int((M1/sizeM1)*(N1/sizeN1))
    
    Emax1 = torch.zeros((N_iter))
    Emax2 = torch.zeros((N_iter))
    Emax3 = torch.zeros((N_iter))
    
    
    count = 0
    
    # Sliding windows index of level 1
    x1 = 0
    y1 = 0
    # Sliding windows index of level 2
    x2 = 0
    y2 = 0
    # Sliding windows index of level 3
    x3 = 0
    y3 = 0
    
    # Sliding windows limit on horizontal dimension
    Y_limit = N1-sizeN1
    
    while count < N_iter:
        # Get the maximum value of slicing windows over edge maps 
        # in each level
        Emax1[count] = torch.max(E1[x1:x1+sizeM1,y1:y1+sizeN1])
        Emax2[count] = torch.max(E2[:,x2:x2+sizeM2,:,y2:y2+sizeN2])
        Emax3[count] = torch.max(E3[:,x3:x3+sizeM3,:,y3:y3+sizeN3])
        
        # if sliding windows ends horizontal direction
        # move along vertical direction and resets horizontal
        # direction
        if y1 == Y_limit:
            x1 = x1 + sizeM1
            y1 = 0
            
            x2 = x2 + sizeM2
            y2 = 0
            
            x3 = x3 + sizeM3
            y3 = 0
            
            count += 1
        
        # windows moves along horizontal dimension
        else:
                
            y1 = y1 + sizeN1
            y2 = y2 + sizeN2
            y3 = y3 + sizeN3
            count += 1
    
    # Step 3
    EdgePoint1 = Emax1 > threshold;
    EdgePoint2 = Emax2 > threshold;
    EdgePoint3 = Emax3 > threshold;
    
    # Rule 1 Edge Pojnts
    EdgePoint = EdgePoint1 + EdgePoint2 + EdgePoint3
    
    n_edges = EdgePoint.shape[0]
    
    # Rule 2 Dirak-Structure or Astep-Structure
    DAstructure = (Emax1[EdgePoint] > Emax2[EdgePoint]) * (Emax2[EdgePoint] > Emax3[EdgePoint]);
    
    # Rule 3 Roof-Structure or Gstep-Structure
    
    RGstructure = torch.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax1[i] < Emax2[i] and Emax2[i] < Emax3[i]:
            
                RGstructure[i] = 1
                
    # Rule 4 Roof-Structure
    
    RSstructure = torch.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax2[i] > Emax1[i] and Emax2[i] > Emax3[i]:
            
                RSstructure[i] = 1

    # Rule 5 Edge more likely to be in a blurred image 

    BlurC = torch.zeros((n_edges));

    for i in range(n_edges):
    
        if RGstructure[i] == 1 or RSstructure[i] == 1:
        
            if Emax1[i] < threshold:
            
                BlurC[i] = 1                        
        
    # Step 6
    Per = torch.sum(DAstructure)/torch.sum(EdgePoint)
    
    # Step 7
    if (torch.sum(RGstructure) + torch.sum(RSstructure)) == 0:
        
        BlurExtent = 100
    else:
        BlurExtent = torch.sum(BlurC) / (torch.sum(RGstructure) + torch.sum(RSstructure))
    
    return Per, BlurExtent

AttributeError: module 'torch.jit' has no attribute 'sctipt'